## Question 1: **Answer**

In [1]:
import sqlite3
import pandas as pd

In [ ]:
def add_data_to_database(input_data):
    """ Connecting to SQL and creating a database  """
    try:
        data = pd.read_csv(input_data)
        engine = sqlite3.connect('climbers_records.db')
        data.to_sql('climbers', con=engine, index = False)
        engine.close()
    except Exception as e:
        print(f"Error:{e}")

# Loading data
data = "/content/climbers.csv"
add_data_to_database(data)

In [3]:
# Loading the extension
%load_ext sql

In [4]:
# Using the extension to connect to database
%sql sqlite:///climbers_records.db

In [5]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM climbers
LIMIT 5;

 * sqlite:///climbers_records.db
Done.


Id,Climber Name,Nationality,Successful Ascents,Failed Attempts,Date of First Ascent,Date of Last Ascent
1101,Edmund Hillary,New Zealand,1,0,29 05 1953,29 05 1953
1102,Tenzing Norgay,Nepal,1,0,29 05 1953,29 05 1953
1103,Reinhold Messner,Italy,2,1,8 05 1978,8 05 1978
1104,Junko Tabei,Japan,1,1,16 05 1975,16 05 1975
1105,Apa Sherpa,Nepal,21,4,10 05 1990,11 05 2011


## Question 2: **Answer**

In [7]:
%%sql
CREATE TABLE IF NOT EXISTS climbers_copy AS
SELECT *
FROM climbers;

 * sqlite:///climbers_records.db
Done.


[]

## Questions 3: **Answer**

In [8]:
%%sql
ALTER TABLE climbers_copy
    RENAME COLUMN 'Climber Name' TO 'climber_name';

ALTER TABLE climbers_copy
    RENAME COLUMN 'Successful Ascents' TO 'successful_ascents';

ALTER TABLE climbers_copy
    RENAME COLUMN 'Failed Attempts' TO 'failed_attempts';

ALTER TABLE climbers_copy
    RENAME COLUMN 'Date of First Ascent' TO 'date_of_first_ascent';

ALTER TABLE climbers_copy
    RENAME COLUMN 'Date of Last Ascent' TO 'date_of_last_ascent';

 * sqlite:///climbers_records.db
Done.
Done.
Done.
Done.
Done.


[]

## Question 4: **Answer**

In [9]:
%%sql
SELECT
    Id,
    printf('%20s', climber_name) AS climber_name,
    printf('%20s', COUNT(*)) AS duplicate_count
FROM climbers_copy
GROUP BY Id, climber_name
HAVING COUNT(*) > 1;

 * sqlite:///climbers_records.db
Done.


Id,climber_name,duplicate_count
1103,Reinhold Messner,2
1104,Junko Tabei,3
1105,Apa Sherpa,2
1106,Phurba Tashi,2
1107,Sir Edmund Hillary,3


## Question 5: **Answer**

In [10]:
%%sql
-- Step 1: Create a new table with unique rows using DISTINCT
CREATE TABLE unique_values AS
SELECT DISTINCT *
FROM climbers_copy;

 * sqlite:///climbers_records.db
Done.


[]

In [11]:
%%sql
-- Step 2: Drop the original table
DROP TABLE climbers_copy;

 * sqlite:///climbers_records.db
Done.


[]

In [12]:
%%sql
-- Step 3: Rename the new table to the original table name
ALTER TABLE unique_values RENAME TO climbers_copy;

 * sqlite:///climbers_records.db
Done.


[]

In [13]:
%%sql
SELECT
    COUNT(*) AS total_duplicates
FROM (
    SELECT
        Id,
        climber_name,
        COUNT(*) AS duplicate_count
    FROM climbers_copy
    GROUP BY Id, climber_name
    HAVING COUNT(*) > 1
) AS duplicates;

 * sqlite:///climbers_records.db
Done.


total_duplicates
0


## Question 6: **Answer**

In [14]:
%%sql
UPDATE climbers_copy
SET date_of_first_ascent = CASE
                    -- Convert 'DD/MM/YYYY' to 'YYYY-MM-DD'
                    WHEN date_of_first_ascent LIKE '__ __ ____'
                THEN SUBSTR(date_of_first_ascent, 7, 4) || '-' ||
                SUBSTR(date_of_first_ascent, 4, 2) || '-' ||
                SUBSTR(date_of_first_ascent, 1, 2)

                -- Convert 'D/MM/YYYY' to 'YYYY-MM-DD'
                    WHEN date_of_first_ascent LIKE '_ __ ____'
                THEN SUBSTR(date_of_first_ascent, 6, 4) || '-' ||
                SUBSTR(date_of_first_ascent, 3, 2) || '-0' ||
                SUBSTR(date_of_first_ascent, 1, 2)
                ELSE date_of_first_ascent
                END;  -- Leave unchanged

 * sqlite:///climbers_records.db
13 rows affected.


[]

In [15]:
%%sql
UPDATE climbers_copy
SET date_of_first_ascent = DATE(date_of_first_ascent);

 * sqlite:///climbers_records.db
13 rows affected.


[]

In [16]:
%%sql
SELECT COUNT(*) AS failed_to_convert
FROM climbers_copy
WHERE date_of_first_ascent IS NULL;

 * sqlite:///climbers_records.db
Done.


failed_to_convert
0


In [17]:
%%sql
UPDATE climbers_copy
SET date_of_last_ascent = CASE
                    -- Convert 'DD/MM/YYYY' to 'YYYY-MM-DD'
                    WHEN date_of_last_ascent LIKE '__ __ ____'
                THEN SUBSTR(date_of_last_ascent, 7, 4) || '-' ||
                SUBSTR(date_of_last_ascent, 4, 2) || '-' ||
                SUBSTR(date_of_last_ascent, 1, 2)

                -- Convert 'D/MM/YYYY' to 'YYYY-MM-DD'
                    WHEN date_of_last_ascent LIKE '_ __ ____'
                THEN SUBSTR(date_of_last_ascent, 6, 4) || '-' ||
                SUBSTR(date_of_last_ascent, 3, 2) || '-0' ||
                SUBSTR(date_of_last_ascent, 1, 2)

                ELSE date_of_last_ascent
                END;  -- Leave unchanged

 * sqlite:///climbers_records.db
13 rows affected.


[]

In [18]:
%%sql
UPDATE climbers_copy
SET date_of_last_ascent = DATE(date_of_last_ascent);

 * sqlite:///climbers_records.db
13 rows affected.


[]

In [19]:
%%sql
SELECT COUNT(*) AS failed_to_convert
FROM climbers_copy
WHERE date_of_last_ascent IS NULL;

 * sqlite:///climbers_records.db
Done.


failed_to_convert
0


In [20]:
%%sql
SELECT * FROM climbers_copy;

 * sqlite:///climbers_records.db
Done.


Id,climber_name,Nationality,successful_ascents,failed_attempts,date_of_first_ascent,date_of_last_ascent
1101,Edmund Hillary,New Zealand,1,0,1953-05-29,1953-05-29
1102,Tenzing Norgay,Nepal,1,0,1953-05-29,1953-05-29
1103,Reinhold Messner,Italy,2,1,1978-05-08,1978-05-08
1104,Junko Tabei,Japan,1,1,1975-05-16,1975-05-16
1105,Apa Sherpa,Nepal,21,4,1990-05-10,2011-05-11
1106,Phurba Tashi,Nepal,21,5,1990-05-10,2014-05-23
1107,Sir Edmund Hillary,New Zealand,1,0,1953-05-29,1953-05-29
1108,Tenzing Norgay,Nepal,1,0,1953-05-29,1953-05-29
1109,Reinhold Messner,Italy,2,1,1978-05-08,1978-05-08
1111,Apa Sherpa,Nepal,21,4,1990-05-10,2011-05-11


## Question 7: **Answer**

In [ ]:
%%sql
UPDATE climbers_copy
SET climber_name = REPLACE(climber_name, 'Sir', '')
WHERE climber_name LIKE 'Sir%';

 * sqlite:///climbers_records.db
1 rows affected.


[]

In [ ]:
%%sql
SELECT COUNT(*) AS count_of_sirs
FROM climbers_copy
WHERE climber_name LIKE 'Sir%';

 * sqlite:///climbers_records.db
Done.


count_of_sirs
0


In [23]:
# %%sql
# DROP TABLE climbers_copy;